In [1]:
# Install notebook dependencies
import sys

#!{sys.executable} -m pip install itk


In [2]:
# Import notebook dependencies
import os
import requests
import shutil

import itk
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib


In [3]:
# Retrieve hosted sample data from data.kitware.com
files_and_hashes = (
    (
        "data/CCFv3_average_template_25_mm_ASL.nii.gz",
        "e29e42a5bda5d1fa75baddc02d2625a87d1d712e9dad34aacce5242b684f6cad590f0e5bde78e8ae4b839e62134a703bd65379a465669afb0189f72b0b08ef71",
    ),
    (
        "data/DzZ_T1.nii.gz",
        "d0b312ad1d38041574de50eb9b1df93bde8dc9b6722d9f4d20e3ed9ab7e4372cd4fa36b6307cfa34edee345805d969cadc21d8e012b0f71bf2006a2e52a7d2bd",
    ),
)

os.makedirs("data", exist_ok=True)

for idx in range(len(files_and_hashes)):
    if os.path.exists(files_and_hashes[idx][0]):
        print(f"{files_and_hashes[idx][0]} already exists.")
    else:
        download_url = f"https://data.kitware.com/api/v1/file/hashsum/sha512/{files_and_hashes[idx][1]}/download"
        print(f"Downloading {download_url} into {files_and_hashes[idx][0]}")
        response = requests.get(download_url, stream=True)
        with open(files_and_hashes[idx][0], "wb") as fp:
            response.raw.decode_content = True
            shutil.copyfileobj(response.raw, fp)


data/CCFv3_average_template_25_mm_ASL.nii.gz already exists.
data/DzZ_T1.nii.gz already exists.


In [4]:
# returns a 4x4 affine matrix for IJK to XYZ mapping
# compatible with nibabel
#
# code taken from TorchIO and simplified
# https://github.com/fepegar/torchio/blob/1bbf99e90cd06112c092a1fc227dedd5deb256ba/torchio/data/io.py#L344-L399
def get_ras_affine_from_itk(itk_image) -> np.ndarray:
    spacing = np.array(itk_image.GetSpacing())
    direction_lps = np.array(itk_image.GetDirection())
    origin_lps = np.array(itk_image.GetOrigin())
    rotation_lps = direction_lps.reshape(3, 3)
    
    FLIPXY_33 = np.diag([-1, -1, 1])
    rotation_ras = np.dot(FLIPXY_33, rotation_lps)
    rotation_ras_zoom = rotation_ras * spacing
    translation_ras = np.dot(FLIPXY_33, origin_lps)
    affine = np.eye(4)
    affine[:3, :3] = rotation_ras_zoom
    affine[:3, 3] = translation_ras
    return affine

# compatible with nibabel and TorchIO
def get_itk_metadata_from_ras_affine(affine: np.ndarray):
    # From https://github.com/nipy/nibabel/blob/master/nibabel/orientations.py
    rotation_zoom = affine[:3, :3]
    spacing = np.sqrt(np.sum(rotation_zoom * rotation_zoom, axis=0))
    direction_ras = rotation_zoom / spacing
    origin_ras = affine[:3, 3]

    FLIPXY_33 = np.diag([-1, -1, 1])
    origin_lps = np.dot(FLIPXY_33, origin_ras)
    direction_lps = np.dot(FLIPXY_33, direction_ras)

    return origin_lps, spacing, direction_lps


In [5]:
# Load the images using both ITK and NiBabel
image = itk.imread(files_and_hashes[1][0])
img = nib.load(files_and_hashes[1][0])


In [6]:
# Show spatial metadata using ITK's conventions
origin, spacing, direction = get_itk_metadata_from_ras_affine(img.affine)
print("Metadata from NiBabel image:")
print(f"origin: {origin}")
print(f"spacing: {spacing}")
print(f"direction:\n{direction}")

print("\nMetadata from ITK image:")
print(f"origin: {image.GetOrigin()}")
print(f"spacing: {image.GetSpacing()}")
print(f"direction:\n{image.GetDirection()}")


Metadata from NiBabel image:
origin: [  26.10700035 -141.74099731  231.56100464]
spacing: [0.68359424 0.68359371 4.40000102]
direction:
[[ 1.77930019e-04  2.54092027e-03 -9.99996756e-01]
 [ 9.90240643e-01 -1.39368011e-01 -1.77930023e-04]
 [-1.39367990e-01 -9.90237396e-01 -2.54092023e-03]]

Metadata from ITK image:
origin: itkPointD3 ([26.107, -141.741, 231.561])
spacing: itkVectorD3 ([0.683594, 0.683594, 4.4])
direction:
itkMatrixD33 ([[0.00017793001938806862, 0.0025409202672601935, -0.999996756027381], [0.9902406432717898, -0.1393680107595337, -0.0001779300230815434], [-0.13936799041953593, -0.9902373964363929, -0.0025409202311968138]])


In [7]:
# Show spatial metadata using NiBabel's conventions
np.set_printoptions(precision=3, suppress=True)

print(f"img.affine:\n{img.affine}\n")
print(f"get_ras_affine_from_itk(image):\n{get_ras_affine_from_itk(image)}")

# print(f"\nimage: {image}")  # Display all, long and ugly
# print(f"\nimg: {img}")  # Display all, long and ugly


img.affine:
[[ -0.     -0.002   4.4   -26.107]
 [ -0.677   0.095   0.001 141.741]
 [ -0.095  -0.677  -0.011 231.561]
 [  0.      0.      0.      1.   ]]

get_ras_affine_from_itk(image):
[[ -0.     -0.002   4.4   -26.107]
 [ -0.677   0.095   0.001 141.741]
 [ -0.095  -0.677  -0.011 231.561]
 [  0.      0.      0.      1.   ]]
